This file uses the premade vector data frame that Rishabh created (for computational efficiency). Not generating vectors manually in this file. Vectors being used will be consistent.

Also, in this file everything is being run on all nodes/vectors (no subgraphs)

TRAINING

In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import pandas as pd
import numpy

import math

import os
import random
# from gensim.models import Word2Vec
import networkx as nx
import seaborn as sns

# from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, accuracy_score, roc_curve, auc, log_loss, RocCurveDisplay


In [2]:
node_df = pd.read_csv("2024_05_05_meta_node_data.csv")
edge_df = pd.read_csv("2024_04_24_meta_edge_sourcetarget_disease_pub_pli_reciprocal.csv")
vector_df = pd.read_csv("df_node_embeddings_dim128_p0.5_q0.5_walk20_n_walks8_wlks_p_node100_cntxt10 (1).csv")
geneGraph = nx.DiGraph()
%run vizfunctions.ipynb
node_df.set_index('id',inplace=True)
node_df.head()

/var/folders/6q/0qnxsny943385fwzz738241c0000gn/T/ipykernel_36948/3282144020.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  edge_df = pd.read_csv("2024_04_24_meta_edge_sourcetarget_disease_pub_pli_reciprocal.csv")


,symbol,disease_assoc_cat,publication_count,ortholog_count,all_ortho_count
id,,,,,
7414,VCL,disease_assoc,279,5,"{'hsapien': {'entrezid': 7414, 'symbol': 'VCL'..."
4626,MYH8,disease_assoc,36,4,"{'hsapien': {'entrezid': 4626, 'symbol': 'MYH8..."
9722,NOS1AP,disease_assoc,141,5,"{'hsapien': {'entrezid': 9722, 'symbol': 'NOS1..."
9891,NUAK1,non_disease_assoc,70,5,"{'hsapien': {'entrezid': 9891, 'symbol': 'NUAK..."
81788,NUAK2,disease_assoc,46,2,"{'hsapien': {'entrezid': 81788, 'symbol': 'NUA..."


In [3]:
# Removing non_omim nodes
all_node_ids = []
for id in vector_df["id"]:
    all_node_ids.append(id)

int_node_ids = []
non_omim_ids = []
for id in all_node_ids:
    if node_df.loc[id]["disease_assoc_cat"] == "disease_assoc" or node_df.loc[id]["disease_assoc_cat"] == "non_disease_assoc":
        int_node_ids.append(id)
    elif node_df.loc[id]["disease_assoc_cat"] == "non_omim":
        non_omim_ids.append(id)

print(len(int_node_ids))
        

16220


In [4]:
# Removing non_omim nodes from df
non_omim_rows = vector_df[vector_df["id"].isin(non_omim_ids)]

training_vector_df = vector_df.drop(non_omim_rows.index)

training_vector_df.head()
print(len(training_vector_df))

16220


In [6]:
vector_df.head()

In [5]:
# Data preprocessing

# Creating a dataset with just the genes, classification, and vectors
# vector_data = pd.DataFrame(vectors)
disease_assoc_cat = numpy.array([node_df.loc[node_id]['disease_assoc_cat'] for node_id in int_node_ids])
publication_count = numpy.array([node_df.loc[node_id]['publication_count'] for node_id in int_node_ids])
ortholog_count = numpy.array([node_df.loc[node_id]['ortholog_count'] for node_id in int_node_ids])
all_ortho_count = numpy.array([node_df.loc[node_id]['all_ortho_count'] for node_id in int_node_ids])


training_vector_df['gene'] = int_node_ids
# vector_data['publication count'] = publication_count
# vector_data['ortholog_count'] = ortholog_count
# vector_data['all_ortho_count'] = all_ortho_count
training_vector_df['disease_assoc_cat'] = disease_assoc_cat

training_vector_df.set_index('gene',inplace=True)

# Creating features
X = training_vector_df.iloc[:, :-1].values    # Taking vectors as input
y = training_vector_df['disease_assoc_cat'].values  # Taking disease association as output

# Splitting data
X_training, X_test, y_training, y_test = train_test_split(X, y, train_size = 0.9, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, train_size = 0.8, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
label_mapping = {
    'non_disease_assoc': 0,
    'disease_assoc': 1,
    'non_omim': 2
}

y_train= [label_mapping[value] for value in y_train]
y_val= [label_mapping[value] for value in y_val]
y_test= [label_mapping[value] for value in y_test]


In [8]:
# Linear SVC
# Initiatlize classifier
svm_model = LinearSVC()

# Perform cross-validation
linear_cv_scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation accuracies:", linear_cv_scores)
print("Mean cross-validation accuracy:", linear_cv_scores.mean())

# Train classifier
svm_model.fit(X_train_scaled, y_train)

# Evaluating classifier
linear_accuracy = svm_model.score(X_test_scaled, y_test)
print(f'Accuracy: {linear_accuracy}')

# Confusion matrix
y_pred = svm_model.predict(X_test_scaled)
linear_cm = confusion_matrix(y_test, y_pred)
print(linear_cm)

lin_svc_train_acc = svm_model.score(X_train, y_train)
print("Training accuracy:", lin_svc_train_acc)

#    vector size       edges    nodes       other features           accuracy               (geneGraph, 50, 10, 0.5, 0.5)
#         128         100000   ~9400            none            0.6232114467408585
#         128         100000   ~9400    pub_count, ortho count  0.3783783783783784
#         512         100000   ~9400            none            0.6063492063492063
#         512         100000   ~9400    pub_count, ortho count  0.3904761904761905
#         64          100000   ~9400            none            0.617879746835443
#         64          100000   ~9400    pub_count, ortho count  0.35917721518987344
#         256         100000   ~9400            none            0.6232227488151659
#         32          100000   ~9400            none            0.6195393169181891


#    vector size       edges      nodes       random walk               accuracy               
#         128          100000     ~9400     (50, 10, 0.5, 0.5)       0.6232114467408585
#         256          100000     ~9400     (50, 10, 0.5, 0.5)       0.6232227488151659
#         512          100000     ~9400     (50, 10, 0.5, 0.5)       0.6063492063492063
#         64           100000     ~9400     (50, 10, 0.5, 0.5)       0.617879746835443

#         64           100000     ~9400     (50, 10, 0.8, 0.4)       0.6283676703645008
#         128          100000     ~9400     (50, 10, 0.8, 0.4)       0.6313291139240507
#         256          100000     ~9400     (50, 10, 0.8, 0.4)       0.6313291139240507
#         512          100000     ~9400     (50, 10, 0.8, 0.4)       0.6006314127861089

#         64           100000     ~9400     (50, 10, 0.4, 0.8)       0.6304868316041501
#         128          100000     ~9400     (50, 10, 0.4, 0.8)       0.6282051282051282
#         256          100000     ~9400     (50, 10, 0.4, 0.8)       0.608
#         512          100000     ~9400     (50, 10, 0.4, 0.8)       0.6142172523961661

#         512          100000     ~9400     (50, 10, 1, 0.2)         0.629222309505106

# ~2 seconds to run

Cross-validation accuracies: [0.70376712 0.70119863 0.70119863 0.70119863 0.70633562 0.70376712
 0.70547945 0.70633562 0.70437018 0.70094259]
Mean cross-validation accuracy: 0.7034593589698442
Accuracy: 0.7071516646115906
[[   2  473]
 [   2 1145]]
Training accuracy: 0.7039732830964206


In [9]:
# Kernel SVC

# Initialize classifier
kernelsvc_model = SVC()

# Perform cross-validation
kernel_cv_scores = cross_val_score(kernelsvc_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation accuracies:", kernel_cv_scores)
print("Mean cross-validation accuracy:", kernel_cv_scores.mean())

# Train classifier
kernelsvc_model.fit(X_train_scaled, y_train)

# Evaluating classifier
kernel_accuracy = kernelsvc_model.score(X_test_scaled, y_test)
print(f'Accuracy: {kernel_accuracy}')

# Confusion matrix
y_pred = kernelsvc_model.predict(X_test_scaled)
kernel_cm = confusion_matrix(y_test, y_pred)
print(kernel_cm)

# Training accuracy
svc_train_acc = kernelsvc_model.score(X_train, y_train)
print("Training accuracy:", svc_train_acc)
#~2 min to run

Cross-validation accuracies: [0.70462329 0.70376712 0.70376712 0.70376712 0.70376712 0.70462329
 0.70462329 0.70376712 0.70351328 0.70437018]
Mean cross-validation accuracy: 0.7040588941320092
Accuracy: 0.7065351418002466
[[   0  475]
 [   1 1146]]
Training accuracy: 0.7039732830964206


In [10]:
# Random forest classifier

# Initiatlize classifier
random_forest_classifier = RandomForestClassifier()

# Perform cross-validation
rfc_cv_scores = cross_val_score(random_forest_classifier, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation accuracies:", rfc_cv_scores)
print("Mean cross-validation accuracy:", rfc_cv_scores.mean())

# Train classifier
random_forest_classifier.fit(X_train_scaled, y_train)

# Evaluating classifier
rfc_accuracy = random_forest_classifier.score(X_test_scaled, y_test)
print(f'Accuracy: {rfc_accuracy}')

# Confusion matrix
y_pred = random_forest_classifier.predict(X_test_scaled)
rfc_cm = confusion_matrix(y_test, y_pred)
print(rfc_cm)

# Training accuracy
rfc_training_acc = random_forest_classifier.score(X_train, y_train)
print("Training accuracy:", rfc_training_acc)
#~4 min to run

Cross-validation accuracies: [0.6994863  0.6994863  0.69691781 0.70291096 0.70119863 0.70119863
 0.69520548 0.70376712 0.69837189 0.70008569]
Mean cross-validation accuracy: 0.699862881642427
Accuracy: 0.7127003699136868
[[  26  449]
 [  17 1130]]
Training accuracy: 0.7039732830964206


In [11]:
# k-nearest neighbors classifier

# Initialize classifier
KNN = KNeighborsClassifier()

# Perform cross-validation
knn_cv_scores = cross_val_score(KNN, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation accuracies:", knn_cv_scores)
print("Mean cross-validation accuracy:", knn_cv_scores.mean())

# Train classifier
KNN.fit(X_train_scaled, y_train)

# Evaluating classifier
knn_accuracy = KNN.score(X_test_scaled, y_test)
print(f'Accuracy: {knn_accuracy}')

# Confusion matrix
y_pred = KNN.predict(X_test_scaled)
knn_cm = confusion_matrix(y_test, y_pred)
print(knn_cm)

# Training accuracy
knn_training_acc = KNN.score(X_train, y_train)
print("Training accuracy:", knn_training_acc)
#~2 min to run



Cross-validation accuracies: [0.63613014 0.65496575 0.64212329 0.61986301 0.65496575 0.62243151
 0.65410959 0.63184932 0.63581834 0.64524422]
Mean cross-validation accuracy: 0.6397500909720512
Accuracy: 0.6436498150431565
[[ 91 384]
 [194 953]]
Training accuracy: 0.7039732830964206


All training stuff is above this point ---------------------


PREDICTIONS ON NONDISEASE GENES

In [41]:
# Isolating non_disease nodes
all_node_ids = []
for id in vector_df["id"]:
    all_node_ids.append(id)

int_node_ids = []
non_disease_ids = []
for id in all_node_ids:
    if node_df.loc[id]["disease_assoc_cat"] == "disease_assoc" or node_df.loc[id]["disease_assoc_cat"] == "non_omim":
        int_node_ids.append(id)
    elif node_df.loc[id]["disease_assoc_cat"] == "non_disease_assoc":
        non_disease_ids.append(id)

print(len(int_node_ids))

# Isolating non-disease node embeddings in a df
unknown_rows = vector_df[vector_df["id"].isin(int_node_ids)]

unknown_vector_df = vector_df.drop(unknown_rows.index)

print(len(unknown_vector_df))

SyntaxError: invalid syntax (43216904.py, line 1)

In [42]:
# Predictions on nondisease genes

# Linear SVM Predictions
y_pred_lin_svm = svm_model.predict(unknown_vector_df)
y_pred_lin_svm = list(y_pred_lin_svm)
# print(y_pred_lin_svm)


# Kernel SVM Predictions
y_pred_ker_svm = kernelsvc_model.predict(unknown_vector_df)
y_pred_ker_svm = list(y_pred_ker_svm)
# print(y_pred_ker_svm)


# KNN Predictions
y_pred_knn = KNN.predict(unknown_vector_df)
y_pred_knn = list(y_pred_knn)
# print(y_pred_knn)


# RFC Predictions
y_pred_rfc = random_forest_classifier.predict(unknown_vector_df)
y_pred_rfc = list(y_pred_rfc)
# print(y_pred_rfc)

/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/envname/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearSVC was fitted without feature names
  warnings.warn(
/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/envname/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


['non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease

/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/envname/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


['non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease

/Users/anusha/Desktop/Neuroinformatics_Internship/HopkinsGeneClassificationProj/envname/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


['non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease_assoc', 'non_disease

In [49]:
for item in y_pred_rfc:
    if item == 'disease_assoc':
        print(item)

In [61]:
#KNN

# Predictions
y_train_pred_knn = KNN.predict(X_train)
y_val_pred_knn = KNN.predict(X_val)
y_test_pred_knn = KNN.predict(X_test)

# Probabilities for ROC curve
y_train_prob_knn = KNN.predict_proba(X_train)[:, 1]
y_val_prob_knn = KNN.predict_proba(X_val)[:, 1]
y_test_prob_knn = KNN.predict_proba(X_test)[:, 1]

# 1. Accuracy and F1 Score
train_acc_knn = accuracy_score(y_train, y_train_pred_knn)
val_acc_knn = accuracy_score(y_val, y_val_pred_knn)
test_acc_knn = accuracy_score(y_test, y_test_pred_knn)

train_f1_knn = f1_score(y_train, y_train_pred_knn)
val_f1_knn = f1_score(y_val, y_val_pred_knn)
test_f1_knn = f1_score(y_test, y_test_pred_knn)

# Log Loss (for Loss plot)
train_loss_knn = log_loss(y_train, y_train_prob_knn)
val_loss_knn = log_loss(y_val, y_val_prob_knn)
test_loss_knn = log_loss(y_test, y_test_prob_knn)

In [58]:
#RFC

# Predictions
y_train_pred_rfc = random_forest_classifier.predict(X_train)
y_val_pred_rfc = random_forest_classifier.predict(X_val)
y_test_pred_rfc = random_forest_classifier.predict(X_test)

# Probabilities for ROC curve
y_train_prob_rfc = random_forest_classifier.predict_proba(X_train)[:, 1]
y_val_prob_rfc = random_forest_classifier.predict_proba(X_val)[:, 1]
y_test_prob_rfc = random_forest_classifier.predict_proba(X_test)[:, 1]

# 1. Accuracy and F1 Score
train_acc_rfc = accuracy_score(y_train, y_train_pred_rfc)
val_acc_rfc = accuracy_score(y_val, y_val_pred_rfc)
test_acc_rfc = accuracy_score(y_test, y_test_pred_rfc)

train_f1_rfc = f1_score(y_train, y_train_pred_rfc)
val_f1_rfc = f1_score(y_val, y_val_pred_rfc)
test_f1_rfc = f1_score(y_test, y_test_pred_rfc)

# Log Loss (for Loss plot)
train_loss_rfc = log_loss(y_train, y_train_prob_rfc)
val_loss_rfc = log_loss(y_val, y_val_prob_rfc)
test_loss_rfc = log_loss(y_test, y_test_prob_rfc)

In [59]:
# 2. Plot Bar Graph for Loss, Accuracy, F1 Score
def plot_metrics_bar(train_acc, val_acc, test_acc, train_f1, val_f1, test_f1, train_loss, val_loss, test_loss):
    metrics = ['Accuracy', 'F1 Score', 'Loss']
    
    # Values for each metric
    train_metrics = [train_acc, train_f1, train_loss]
    val_metrics = [val_acc, val_f1, val_loss]
    test_metrics = [test_acc, test_f1, test_loss]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns
    
    # Plot each metric as bar plots
    for i, metric in enumerate(metrics):
        axes[i].bar(['Train', 'Validation', 'Test'], 
                    [train_metrics[i], val_metrics[i], test_metrics[i]], 
                    color=['blue', 'orange', 'green'])
        axes[i].set_title(f'{metric} Bar Plot')
        axes[i].set_ylabel(metric)
        axes[i].set_xlabel('Dataset Split')
    
    plt.tight_layout()
    plt.show()

plot_metrics_bar(train_acc_rfc, val_acc_rfc, test_acc_rfc, train_f1_rfc, val_f1_rfc, test_f1_rfc, train_loss_rfc, val_loss_rfc, test_loss_rfc)


# 3. Confusion Matrix
def plot_confusion_matrices(y_train, y_train_pred, y_val, y_val_pred, y_test, y_test_pred):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns
    splits = ['Training', 'Validation', 'Test']
    preds = [(y_train, y_train_pred), (y_val, y_val_pred), (y_test, y_test_pred)]
    
    for i, (y_true, y_pred) in enumerate(preds):
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
        axes[i].set_title(f'{splits[i]} Confusion Matrix')
        axes[i].set_xlabel('Predicted Label')
        axes[i].set_ylabel('True Label')
    
    plt.tight_layout()
    plt.show()

plot_confusion_matrices(y_train, y_train_pred_rfc, y_val, y_val_pred_rfc, y_test, y_test_pred_rfc)

# 4. ROC Curves and Optimal Threshold Calculation
def calculate_roc_curve(y_true, y_prob):
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    return fpr, tpr, thresholds, roc_auc


results = {}
for split, (y_true, y_prob) in zip(['train', 'val', 'test'], [(y_train, y_train_prob_rfc), (y_val, y_val_prob_rfc), (y_test, y_test_prob_rfc)]):
    fpr, tpr, thresholds, roc_auc = calculate_roc_curve(y_true, y_prob)
    results[f'{split}_fpr'] = fpr
    results[f'{split}_tpr'] = tpr
    results[f'{split}_thresholds'] = thresholds
    results[f'{split}_roc_auc'] = roc_auc

# Plot ROC Curves
def plot_roc_curves(results):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Create 1 row with 3 subplots
    titles = ["Training ROC Curve", "Validation ROC Curve", "Test ROC Curve"]
    
    for i, metric in enumerate(['train', 'val', 'test']):
        fpr = results[f'{metric}_fpr']
        tpr = results[f'{metric}_tpr']
        roc_auc = results[f'{metric}_roc_auc']
        thresholds = results[f'{metric}_thresholds']
        
        # Compute the Youden's J statistic for each threshold
        youdens_j = tpr - fpr
        optimal_idx = numpy.argmax(youdens_j)  # Get the index of the max J statistic
        optimal_threshold = thresholds[optimal_idx]
        optimal_fpr = fpr[optimal_idx]
        optimal_tpr = tpr[optimal_idx]
        
        # Plot the ROC curve
        RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot(ax=axes[i])
        axes[i].set_title(f'{metric.capitalize()} ROC Curve')

        # Highlight the optimal threshold
        axes[i].scatter(optimal_fpr, optimal_tpr, color='red', label=f'Optimal Threshold: {optimal_threshold:.2f}')
        axes[i].annotate(f'Threshold: {optimal_threshold:.2f}\nTPR: {optimal_tpr:.2f}, FPR: {optimal_fpr:.2f}',
                         xy=(optimal_fpr, optimal_tpr), xycoords='data',
                         xytext=(-50, 40), textcoords='offset points',
                         arrowprops=dict(facecolor='black', shrink=0.05),
                         fontsize=10, color='black')

        axes[i].legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()

plot_roc_curves(results)

In [42]:
# 2. Plot Bar Graph for Loss, Accuracy, F1 Score
def plot_metrics_bar(train_acc, val_acc, test_acc, train_f1, val_f1, test_f1, train_loss, val_loss, test_loss):
    metrics = ['Accuracy', 'F1 Score', 'Loss']
    
    # Values for each metric
    train_metrics = [train_acc, train_f1, train_loss]
    val_metrics = [val_acc, val_f1, val_loss]
    test_metrics = [test_acc, test_f1, test_loss]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns
    
    # Plot each metric as bar plots
    for i, metric in enumerate(metrics):
        axes[i].bar(['Train', 'Validation', 'Test'], 
                    [train_metrics[i], val_metrics[i], test_metrics[i]], 
                    color=['blue', 'orange', 'green'])
        axes[i].set_title(f'{metric} Bar Plot')
        axes[i].set_ylabel(metric)
        axes[i].set_xlabel('Dataset Split')
    
    plt.tight_layout()
    plt.show()

plot_metrics_bar(train_acc_knn, val_acc_knn, test_acc_knn, train_f1_knn, val_f1_knn, test_f1_knn, train_loss_knn, val_loss_knn, test_loss_knn)


# 3. Confusion Matrix
def plot_confusion_matrices(y_train, y_train_pred, y_val, y_val_pred, y_test, y_test_pred):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns
    splits = ['Training', 'Validation', 'Test']
    preds = [(y_train, y_train_pred), (y_val, y_val_pred), (y_test, y_test_pred)]
    
    for i, (y_true, y_pred) in enumerate(preds):
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
        axes[i].set_title(f'{splits[i]} Confusion Matrix')
        axes[i].set_xlabel('Predicted Label')
        axes[i].set_ylabel('True Label')
    
    plt.tight_layout()
    plt.show()

plot_confusion_matrices(y_train, y_train_pred_knn, y_val, y_val_pred_knn, y_test, y_test_pred_knn)

# Define a mapping function
def map_values(x):
    mapping = {
            0: 'disease_assoc',
            1: 'non_disease_assoc',
            2: 'non_omim'
        }

# 4. ROC Curves and Optimal Threshold Calculation
def calculate_roc_curve(y_true, y_prob):
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)

    mapped_y_true = y_true.vectorize(map_values)
    return fpr, tpr, thresholds, roc_auc


results = {}
for split, (y_true, y_prob) in zip(['train', 'val', 'test'], [(y_train, y_train_prob_knn), (y_val, y_val_prob_knn), (y_test, y_test_prob_knn)]):
    fpr, tpr, thresholds, roc_auc = calculate_roc_curve(y_true, y_prob)
    results[f'{split}_fpr'] = fpr
    results[f'{split}_tpr'] = tpr
    results[f'{split}_thresholds'] = thresholds
    results[f'{split}_roc_auc'] = roc_auc

# Plot ROC Curves
def plot_roc_curves(results):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Create 1 row with 3 subplots
    titles = ["Training ROC Curve", "Validation ROC Curve", "Test ROC Curve"]
    
    for i, metric in enumerate(['train', 'val', 'test']):
        fpr = results[f'{metric}_fpr']
        tpr = results[f'{metric}_tpr']
        roc_auc = results[f'{metric}_roc_auc']
        thresholds = results[f'{metric}_thresholds']
        
        # Compute the Youden's J statistic for each threshold
        youdens_j = tpr - fpr
        optimal_idx = numpy.argmax(youdens_j)  # Get the index of the max J statistic
        optimal_threshold = thresholds[optimal_idx]
        optimal_fpr = fpr[optimal_idx]
        optimal_tpr = tpr[optimal_idx]
        
        # Plot the ROC curve
        RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot(ax=axes[i])
        axes[i].set_title(f'{metric.capitalize()} ROC Curve')

        # Highlight the optimal threshold
        axes[i].scatter(optimal_fpr, optimal_tpr, color='red', label=f'Optimal Threshold: {optimal_threshold:.2f}')
        axes[i].annotate(f'Threshold: {optimal_threshold:.2f}\nTPR: {optimal_tpr:.2f}, FPR: {optimal_fpr:.2f}',
                         xy=(optimal_fpr, optimal_tpr), xycoords='data',
                         xytext=(-50, 40), textcoords='offset points',
                         arrowprops=dict(facecolor='black', shrink=0.05),
                         fontsize=10, color='black')

        axes[i].legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()

plot_roc_curves(results)

In [14]:
# Bayesian Optimization hyperparameter tuning

# LinearSVC
param_space_linear_svc = {
    'C': (1e-4, 1e+2, 'log-uniform'),  # Regularization parameter
    'max_iter': (100, 1000)             # Maximum number of iterations
}

# Using BayesSearchCV for Bayesian Optimization
linear_opt = BayesSearchCV(svm_model, param_space_linear_svc, n_iter=50, cv=5, n_jobs=-1, random_state=42)

# Fit the model
linear_opt.fit(X_train_scaled, y_train)

# Best parameters and score
print("Linear best parameters:", linear_opt.best_params_)
print("Linear best cross-validation score:", linear_opt.best_score_)

# Evaluate on the test set
y_pred_linear = linear_opt.predict(X_test_scaled)
test_accuracy_linear = accuracy_score(y_test, y_pred_linear)
print("Linear test set accuracy:", test_accuracy_linear)

# ALL CODE IS THE SAME [ONLY MODEL IS DIFFERENT] -- no comments written bc everything is the same
# KernelSVC
param_space_kernel_svc = {
    'C': (1e-4, 1e+2, 'log-uniform'),   # Regularization parameter
    'gamma': (1e-4, 1e+1, 'log-uniform'), # Kernel coefficient
    'kernel': ['linear', 'rbf', 'poly'], # Kernel type
    'degree': (2, 5)                     # Degree for polynomial kernel
}

kernel_opt = BayesSearchCV(kernelsvc_model, param_space_kernel_svc, n_iter=50, cv=5, n_jobs=-1, random_state=42)
kernel_opt.fit(X_train_scaled, y_train)
print("Kernel best parameters:", kernel_opt.best_params_)
print("Kernel best cross-validation score:", kernel_opt.best_score_)
y_pred_kernel = kernel_opt.predict(X_test_scaled)
test_accuracy_kernel = accuracy_score(y_test, y_pred_kernel)
print("Kernel test set accuracy:", test_accuracy_kernel)

# Random Forest Classifier
param_space_random_forest = {
    'n_estimators': (10, 100),           # Number of trees in the forest
    'max_depth': (None, 5, 10, 15),      # Maximum depth of the tree
    'min_samples_split': (2, 10),        # Minimum number of samples required to split an internal node
    'max_features': ['auto', 'sqrt']     # Number of features to consider when looking for the best split
}

rfc_opt = BayesSearchCV(random_forest_classifier, param_space_random_forest, n_iter=50, cv=5, n_jobs=-1, random_state=42)
rfc_opt.fit(X_train_scaled, y_train)
print("RFC best parameters:", rfc_opt.best_params_)
print("RFC best cross-validation score:", rfc_opt.best_score_)
y_pred_rfc = rfc_opt.predict(X_test_scaled)
test_accuracy_rfc = accuracy_score(y_test, y_pred_rfc)
print("RFC test set accuracy:", test_accuracy_rfc)

# KNN
param_space_knn = {
    'n_neighbors': (1, 20),              # Number of neighbors
    'weights': ['uniform', 'distance'],   # Weight function used in prediction
    'metric': ['euclidean', 'manhattan']  # Distance metric
}

knn_opt = BayesSearchCV(KNN, param_space_knn, n_iter=50, cv=5, n_jobs=-1, random_state=42)
knn_opt.fit(X_train_scaled, y_train)
print("KNN best parameters:", knn_opt.best_params_)
print("KNN best cross-validation score:", knn_opt.best_score_)
y_pred_knn = knn_opt.predict(X_test_scaled)
test_accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNN test set accuracy:", test_accuracy_knn)


In [ ]:
# # GridSearch CV hyperparameter tuning
# # defining parameter range 
# param_grid = {'loss':['hinge', 'squared_hinge'],
#               'tol': [1, 0.1, 0.01, 0.001, 0.0001],
#               'C': [0.1, 1, 10, 100, 1000],
#               'max_iter':[1000, 3000, 5000]   
#               }  
  
# grid = GridSearchCV(svm_model, param_grid, refit = True, verbose = 3) 
  
# # fitting the model for grid search 
# grid.fit(X_train, y_train) 

# # print best parameter after tuning 
# print(grid.best_params_) 
  
# # print how our model looks after hyper-parameter tuning 
# print(grid.best_estimator_) 

# grid_predictions = grid.predict(X_test) 
  
# # print classification report 
# print(classification_report(y_test, grid_predictions)) 